In [1]:
import pycaret
from pycaret.regression import *
from pycaret.classification import *
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from tableone import TableOne

In [2]:
pre = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\FDATA_Pre_2B_GAUS_NORM_RESAMPLE000_5Sig_2023-12-17-19-39-33_radiomics_features_ALL.csv", index_col=0) 
pre.set_index('pt_no', inplace=True)
current_pre = pre.columns.tolist()
new_pre = {col: 'pre_' + col for col in current_pre}
pre = pre.rename(columns=new_pre)
pre

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,5.151790,0.020648,2.521641,0.000432,2.845642e+07,9.015553,0.006481,0.170626,0.000521,0.029926
17815426,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,2.745954,0.090834,3.517816,0.006705,1.881134e+05,19.008323,0.002101,5.205460,0.019972,0.033623
20109646,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,3.295786,0.033080,4.901244,0.000994,3.646460e+07,28.580566,0.001058,0.616141,0.005491,0.017261
22320836,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,5.047231,0.026280,4.297276,0.000924,3.283239e+07,15.015660,0.002133,0.337529,0.000162,0.028521
32221501,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,8.966514,0.014361,3.076103,0.002722,1.249836e+06,29.483228,0.001257,3.622199,0.011019,0.020562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,2.694805,13784.733946,8.584567e-02,0.071341,24.932035,3.403854,2.593937,0.073887,2.611643,1.458692,...,1.496849,0.196455,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850
56124435,2.403171,81778.369047,1.490141e-01,0.073894,33.752847,2.604287,2.318845,0.067155,2.343169,1.033921,...,4.021003,0.058208,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413
56131318,1.982925,44892.814442,2.641421e-01,0.050950,20.583522,2.235937,1.905351,0.079641,1.944605,0.941089,...,6.721493,0.040658,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484


In [3]:
ypCR_data = pd.read_excel(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\clinical_immunologic.xlsx", index_col=0) 
ypCR_data.set_index('pt_no', inplace=True)
ypCR_data = ypCR_data[ypCR_data['Pathol_available'] == 1]
ypCR_data

,name,Ineligible,Ineligible_2019,Pathol_available,Hospital,note,base.fu,date.dx,dob,age,...,note.1,other.malig,pre_PDL1_CPS,pre_PDL1_IC,pre_CD3_num_Pos_mm,pre_CD8_num_Pos_mm,post_PDL1_CPS,post_PDL1_IC,post_CD3_num_Pos_mm,post_CD8_num_Pos_mm
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,정종일,0,0,1.0,1,NaN,2016-11-17,2016-10-24,1946-03-05,69.731507,...,NaN,NaN,5.0,5.0,366.0532,51.9522,10.0,10.0,3171.7243,890.4297
17815426,김진영,1,0,1.0,1,NaN,2021-07-30,2021-07-08,1955-01-04,65.660274,...,NaN,NaN,50.0,40.0,637.6734,374.4912,80.0,80.0,1532.5386,1298.4060
20109646,서대현,0,0,1.0,1,NaN,2018-07-09,2018-05-18,1951-06-20,66.134247,...,NaN,NaN,0.5,0.5,60.6479,39.7833,0.0,0.0,100.5308,43.6339
22320836,최흥우,0,0,1.0,1,NaN,2016-06-07,2016-05-18,1947-07-11,67.961644,...,"diaphragmatic hernia reduction (2020/3/17, 201...",NaN,10.0,10.0,608.9312,140.8317,10.0,10.0,1176.5166,1516.6479
32221501,박남훈,0,0,1.0,1,NaN,2010-10-11,2010-09-17,1939-10-16,70.013699,...,NaN,NaN,1.0,1.0,504.0535,64.6015,10.0,10.0,1245.4099,1155.8683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,최정옥,1,0,1.0,1,Previous malignancy: Breast ca (하지만 1995년이라 ok),2021-10-26,2021-09-29,1961-11-27,59.093151,...,NaN,NaN,10.0,5.0,1152.7963,471.6934,100.0,70.0,1477.2669,868.1072
56124435,김정자,1,0,1.0,1,NaN,2021-12-03,2021-11-09,1963-01-13,58.082192,...,NaN,NaN,0.0,0.0,145.0536,31.3657,10.0,10.0,479.3238,389.0902
56131318,홍성배,1,0,1.0,1,NaN,2021-11-23,2021-11-10,1951-05-20,69.542466,...,NaN,NaN,10.0,10.0,5283.4120,5773.6300,50.0,50.0,4583.2514,4448.2354


In [4]:
pre['post_PDL1_CPS'] = ypCR_data['post_PDL1_CPS']
pre

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,0.020648,2.521641,0.000432,2.845642e+07,9.015553,0.006481,0.170626,0.000521,0.029926,10.0
17815426,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,0.090834,3.517816,0.006705,1.881134e+05,19.008323,0.002101,5.205460,0.019972,0.033623,80.0
20109646,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,0.033080,4.901244,0.000994,3.646460e+07,28.580566,0.001058,0.616141,0.005491,0.017261,0.0
22320836,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,0.026280,4.297276,0.000924,3.283239e+07,15.015660,0.002133,0.337529,0.000162,0.028521,10.0
32221501,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,0.014361,3.076103,0.002722,1.249836e+06,29.483228,0.001257,3.622199,0.011019,0.020562,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,2.694805,13784.733946,8.584567e-02,0.071341,24.932035,3.403854,2.593937,0.073887,2.611643,1.458692,...,0.196455,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850,100.0
56124435,2.403171,81778.369047,1.490141e-01,0.073894,33.752847,2.604287,2.318845,0.067155,2.343169,1.033921,...,0.058208,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413,10.0
56131318,1.982925,44892.814442,2.641421e-01,0.050950,20.583522,2.235937,1.905351,0.079641,1.944605,0.941089,...,0.040658,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484,50.0


In [5]:
# bin_labels = [0, 1, 2]
# bins = [-1, 1, 10, float('inf')]
# pre['encoded_PDL1'] = pd.cut(pre['post_PDL1_CPS'], bins=bins, labels=bin_labels)
# pre
conditions = [
    (pre['post_PDL1_CPS'] >= 0) & (pre['post_PDL1_CPS'] < 1),
    (pre['post_PDL1_CPS'] >= 1) & (pre['post_PDL1_CPS'] <= 10),
    (pre['post_PDL1_CPS'] > 10)
]
choices = [0, 1, 2]
pre['encoded_PDL1'] = np.select(conditions, choices, default=np.nan)
pre

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,2.521641,0.000432,2.845642e+07,9.015553,0.006481,0.170626,0.000521,0.029926,10.0,1.0
17815426,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,3.517816,0.006705,1.881134e+05,19.008323,0.002101,5.205460,0.019972,0.033623,80.0,2.0
20109646,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,4.901244,0.000994,3.646460e+07,28.580566,0.001058,0.616141,0.005491,0.017261,0.0,0.0
22320836,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,4.297276,0.000924,3.283239e+07,15.015660,0.002133,0.337529,0.000162,0.028521,10.0,1.0
32221501,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,3.076103,0.002722,1.249836e+06,29.483228,0.001257,3.622199,0.011019,0.020562,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,2.694805,13784.733946,8.584567e-02,0.071341,24.932035,3.403854,2.593937,0.073887,2.611643,1.458692,...,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850,100.0,2.0
56124435,2.403171,81778.369047,1.490141e-01,0.073894,33.752847,2.604287,2.318845,0.067155,2.343169,1.033921,...,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413,10.0,1.0
56131318,1.982925,44892.814442,2.641421e-01,0.050950,20.583522,2.235937,1.905351,0.079641,1.944605,0.941089,...,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484,50.0,2.0


In [6]:
pre.drop(columns=['post_PDL1_CPS'], inplace=True)
pre

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,0.020648,2.521641,0.000432,2.845642e+07,9.015553,0.006481,0.170626,0.000521,0.029926,1.0
17815426,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,0.090834,3.517816,0.006705,1.881134e+05,19.008323,0.002101,5.205460,0.019972,0.033623,2.0
20109646,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,0.033080,4.901244,0.000994,3.646460e+07,28.580566,0.001058,0.616141,0.005491,0.017261,0.0
22320836,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,0.026280,4.297276,0.000924,3.283239e+07,15.015660,0.002133,0.337529,0.000162,0.028521,1.0
32221501,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,0.014361,3.076103,0.002722,1.249836e+06,29.483228,0.001257,3.622199,0.011019,0.020562,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,2.694805,13784.733946,8.584567e-02,0.071341,24.932035,3.403854,2.593937,0.073887,2.611643,1.458692,...,0.196455,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850,2.0
56124435,2.403171,81778.369047,1.490141e-01,0.073894,33.752847,2.604287,2.318845,0.067155,2.343169,1.033921,...,0.058208,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413,1.0
56131318,1.982925,44892.814442,2.641421e-01,0.050950,20.583522,2.235937,1.905351,0.079641,1.944605,0.941089,...,0.040658,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484,2.0


In [7]:
feature = pre.drop(columns='encoded_PDL1')
target = pre['encoded_PDL1']
list_numerical = feature.columns.to_list()
list_numerical

['pre_exponential_firstorder_90Percentile',
 'pre_exponential_firstorder_Energy',
 'pre_exponential_firstorder_Entropy',
 'pre_exponential_firstorder_InterquartileRange',
 'pre_exponential_firstorder_Kurtosis',
 'pre_exponential_firstorder_Maximum',
 'pre_exponential_firstorder_Mean',
 'pre_exponential_firstorder_MeanAbsoluteDeviation',
 'pre_exponential_firstorder_Median',
 'pre_exponential_firstorder_Minimum',
 'pre_exponential_firstorder_Range',
 'pre_exponential_firstorder_RobustMeanAbsoluteDeviation',
 'pre_exponential_firstorder_RootMeanSquared',
 'pre_exponential_firstorder_Skewness',
 'pre_exponential_firstorder_TotalEnergy',
 'pre_exponential_firstorder_Uniformity',
 'pre_exponential_firstorder_Variance',
 'pre_exponential_glcm_Autocorrelation',
 'pre_exponential_glcm_ClusterProminence',
 'pre_exponential_glcm_ClusterShade',
 'pre_exponential_glcm_ClusterTendency',
 'pre_exponential_glcm_Contrast',
 'pre_exponential_glcm_Correlation',
 'pre_exponential_glcm_DifferenceAverage',

In [8]:
feature

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,5.151790,0.020648,2.521641,0.000432,2.845642e+07,9.015553,0.006481,0.170626,0.000521,0.029926
17815426,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,2.745954,0.090834,3.517816,0.006705,1.881134e+05,19.008323,0.002101,5.205460,0.019972,0.033623
20109646,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,3.295786,0.033080,4.901244,0.000994,3.646460e+07,28.580566,0.001058,0.616141,0.005491,0.017261
22320836,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,5.047231,0.026280,4.297276,0.000924,3.283239e+07,15.015660,0.002133,0.337529,0.000162,0.028521
32221501,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,8.966514,0.014361,3.076103,0.002722,1.249836e+06,29.483228,0.001257,3.622199,0.011019,0.020562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,2.694805,13784.733946,8.584567e-02,0.071341,24.932035,3.403854,2.593937,0.073887,2.611643,1.458692,...,1.496849,0.196455,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850
56124435,2.403171,81778.369047,1.490141e-01,0.073894,33.752847,2.604287,2.318845,0.067155,2.343169,1.033921,...,4.021003,0.058208,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413
56131318,1.982925,44892.814442,2.641421e-01,0.050950,20.583522,2.235937,1.905351,0.079641,1.944605,0.941089,...,6.721493,0.040658,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484


In [9]:
# def custom_zscore(column):
#     if column.std() == 0:
#         return (column - column.mean())  # For columns with zero variance, mean-centered values
#     else:
#         return (column - column.mean()) / column.std()

# # Apply custom_zscore to each column
# feature = feature.apply(custom_zscore)
# feature

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import Lasso, LassoCV
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [11]:
# lasso 를 위한 데이터 나누기 
X_train, X_test, y_train, y_test = train_test_split(feature, target, shuffle=True, test_size = 0.2, stratify=target, random_state=2023)

In [12]:
X_train

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
41375060,1.913437,16892.002232,1.343667e-02,0.056566,22.919338,2.044471,1.838553,0.067367,1.869661,0.897939,...,0.000294,0.000260,1.584963e+00,0.000603,4.775943e+06,8.925614,0.009806,0.150564,0.000874,0.048268
37567637,2.086047,9293.345336,8.503110e-01,0.070566,20.438827,2.149139,1.999038,0.078855,2.036572,1.000295,...,5.279460,0.064826,3.139354e+00,0.008647,1.338452e+05,12.762282,0.004291,1.865205,0.011500,0.043976
49087653,1.806664,23083.640462,-3.203427e-16,0.068359,18.502705,1.917878,1.719973,0.070291,1.748438,0.936480,...,4.139382,0.041369,3.973661e+00,0.003222,1.398731e+06,15.029241,0.001964,2.202689,0.006387,0.042920
46695156,1.733236,7984.640391,1.925557e-01,0.045929,266.121365,11.138778,1.728045,0.095784,1.694074,1.048333,...,15.734889,0.028606,4.773557e+00,0.012034,1.648698e+05,1.362353,0.008716,8.345484,0.000758,1.450861
54502130,2.461475,22585.726458,1.653282e-01,0.052003,40.364190,2.730718,2.390674,0.060950,2.413734,1.017602,...,7.130450,0.028599,3.616875e+00,0.004316,8.013075e+05,2.313348,0.013862,0.438290,0.000230,0.180779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,3.179595,42314.189622,3.927374e-01,0.654389,5.508685,5.148650,2.502102,0.398477,2.383120,1.818160,...,4.053328,0.122144,4.423252e+00,0.004315,2.913502e+05,21.288096,0.002779,2.809775,0.007782,0.044334
37509338,2.263698,4558.466574,8.426243e-02,0.086331,3.544149,2.475461,2.189351,0.055713,2.201546,1.975546,...,0.000001,0.000001,-3.203427e-16,0.001053,0.000000e+00,0.000000,1000000.000000,0.000000,0.000000,0.000000
50740947,1.944689,55447.935894,3.301658e-02,0.031311,106.157388,2.114963,1.911616,0.023902,1.917436,1.013016,...,5.286004,0.066648,2.219241e+00,0.001319,1.072499e+07,2.745231,0.015290,0.071754,0.000025,0.200667


In [13]:
X_test

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
42748515,2.201648,39069.292027,2.315166e-01,0.069860,43.469720,2.427748,2.125740,0.054280,2.137784,0.941167,...,0.482977,0.006213,2.584963,0.000695,4.035376e+06,78.144128,0.000833,2.520254,0.013013,0.008291
22320836,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,5.047231,0.026280,4.297276,0.000924,3.283239e+07,15.015660,0.002133,0.337529,0.000162,0.028521
56131813,2.398622,46511.517902,1.060820e-01,0.057122,44.788503,3.312703,2.334439,0.051460,2.349742,1.118705,...,4.053635,0.081890,4.411769,0.004695,1.617504e+06,7.366517,0.005814,0.562301,0.000261,0.044813
37244770,1.901291,83628.209721,5.243291e-03,0.045579,4.274601,2.023777,1.859029,0.029094,1.864095,1.559196,...,0.563424,0.563424,1.879965,0.000372,5.695521e+07,14.985006,0.016845,0.004909,0.000039,0.013383
49344356,1.685784,46081.954347,2.363807e-03,0.042663,11.579530,2.241984,1.604154,0.091589,1.653138,0.857264,...,3.803438,0.028066,3.948859,0.001635,8.351460e+06,16.522003,0.002086,0.693884,0.002348,0.025297
42120720,2.354662,15279.297108,4.372304e-01,0.102524,12.463237,2.819949,2.212760,0.161193,2.307719,0.959301,...,7.333897,0.031752,3.856196,0.008795,1.305989e+05,8.831242,0.002755,6.439021,0.016149,0.088422
51524302,2.448859,60249.579189,3.580777e-02,0.068568,29.175909,3.479093,2.382891,0.047403,2.393265,1.387185,...,3.359322,0.117854,3.295357,0.002075,4.777348e+06,3.209797,0.017463,0.089043,0.000022,0.085111
47451087,1.746566,40093.208534,-3.203427e-16,0.054608,19.606255,1.842178,1.683773,0.054437,1.707863,1.028719,...,3.547951,0.074681,3.085055,0.001844,5.891494e+06,18.962206,0.002407,0.524917,0.002154,0.027593
53004114,2.314032,16411.127041,1.428297e-01,0.081631,17.944753,2.575424,2.225085,0.059852,2.232936,1.355104,...,2.189075,0.124004,3.392747,0.003929,7.447841e+05,1.557459,0.027178,0.208109,0.000118,0.319877


In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train[list_numerical]) 

X_train[list_numerical] = scaler.transform(X_train[list_numerical])

X_test[list_numerical] = scaler.transform(X_test[list_numerical])

In [15]:
X_train

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
41375060,-1.056040,-0.546057,-0.935698,-0.354476,-0.161911,-0.537157,-1.067749,-0.347070,-1.058970,-0.758498,...,-1.240590,-0.922667,-2.099837,-1.068365,0.160132,-0.123043,-0.112509,-0.653653,-0.558266,-0.309178
37567637,-0.527841,-0.790511,1.808028,-0.255055,-0.185794,-0.470398,-0.505671,-0.212172,-0.479077,-0.438568,...,0.617931,-0.297368,-0.329478,1.354422,-0.540842,-0.117686,-0.112509,0.228618,0.653454,-0.334411
49087653,-1.382771,-0.346868,-0.979751,-0.270728,-0.204436,-0.617901,-1.483059,-0.312736,-1.480131,-0.638032,...,0.216568,-0.524542,0.620748,-0.279429,-0.349839,-0.114521,-0.112509,0.402271,0.070435,-0.340620
46695156,-1.607464,-0.832613,-0.348449,-0.430017,2.179736,5.263398,-1.454787,-0.013381,-1.669006,-0.288418,...,4.298744,-0.648143,1.531783,2.374594,-0.536157,-0.133603,-0.112509,3.563056,-0.571456,7.936690
54502130,0.620992,-0.362886,-0.437716,-0.386881,0.006055,-0.099454,0.865980,-0.422430,0.831282,-0.384472,...,1.269568,-0.648209,0.214390,0.049894,-0.440052,-0.132275,-0.112509,-0.505604,-0.631664,0.469856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,2.818484,0.271791,0.307854,3.891071,-0.329547,1.442759,1.256242,3.541132,0.724921,2.117791,...,0.186273,0.257721,1.132806,0.049647,-0.517058,-0.105782,-0.112509,0.714648,0.229489,-0.332305
37509338,0.015781,-0.942835,-0.703493,-0.143093,-0.348463,-0.262262,0.160874,-0.483925,0.094085,2.609726,...,-1.240693,-0.925170,-3.905015,-0.933009,-0.561053,-0.135505,8.888194,-0.731127,-0.657882,-0.592944
50740947,-0.960405,0.694312,-0.871505,-0.533831,0.639539,-0.492196,-0.811854,-0.857484,-0.892987,-0.398807,...,0.620234,-0.279722,-1.377432,-0.852802,1.058458,-0.131672,-0.112509,-0.694206,-0.655022,0.586782


In [16]:
n_alphas = 100
alphas = np.logspace(-4, -1.2, n_alphas)
print(alphas)
lasso_cv = LassoCV(alphas = alphas, random_state=2023, max_iter=1000000, cv=KFold(n_splits=5), verbose=True, n_jobs=-1).fit(X_train, y_train)

[0.0001     0.00010673 0.00011391 0.00012158 0.00012976 0.00013849
 0.00014781 0.00015775 0.00016837 0.0001797  0.00019179 0.0002047
 0.00021847 0.00023317 0.00024886 0.00026561 0.00028348 0.00030256
 0.00032292 0.00034465 0.00036784 0.00039259 0.00041901 0.0004472
 0.0004773  0.00050941 0.00054369 0.00058028 0.00061933 0.000661
 0.00070548 0.00075295 0.00080362 0.0008577  0.00091541 0.00097701
 0.00104275 0.00111292 0.00118781 0.00126774 0.00135305 0.0014441
 0.00154127 0.00164498 0.00175568 0.00187382 0.00199991 0.00213448
 0.00227812 0.00243141 0.00259502 0.00276965 0.00295602 0.00315493
 0.00336723 0.00359381 0.00383564 0.00409375 0.00436922 0.00466323
 0.00497702 0.00531193 0.00566938 0.00605088 0.00645804 0.00689261
 0.00735642 0.00785144 0.00837977 0.00894366 0.00954548 0.01018781
 0.01087336 0.01160504 0.01238595 0.01321941 0.01410896 0.01505836
 0.01607166 0.01715313 0.01830738 0.0195393  0.02085412 0.02225742
 0.02375514 0.02535364 0.02705972 0.02888059 0.03082399 0.03289817


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.7s finished


In [17]:
print('R squared training set', round(lasso_cv.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_cv.score(X_test, y_test)*100, 2))

R squared training set 70.03
R squared test set 21.87


In [18]:
best_alpha = lasso_cv.alpha_
print("Best alpha parameter:", best_alpha)

Best alpha parameter: 0.06309573444801933


In [19]:
feature_list = lasso_cv.feature_names_in_[lasso_cv.coef_ != 0]
print(len(feature_list), feature_list)

37 ['pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis'
 'pre_gradient_ngtdm_Strength'
 'pre_log-sigma-0-5-mm-3D_firstorder_Maximum'
 'pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized'
 'pre_log-sigma-0-5-mm-3D_glszm_ZoneEntropy'
 'pre_log-sigma-1-5-mm-3D_gldm_DependenceVariance'
 'pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized'
 'pre_log-sigma-1-5-mm-3D_glszm_ZoneEntropy'
 'pre_log-sigma-2-0-mm-3D_glszm_SizeZoneNonUniformityNormalized'
 'pre_log-sigma-2-5-mm-3D_glcm_ClusterProminence'
 'pre_log-sigma-2-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis'
 'pre_logarithm_firstorder_Mean'
 'pre_original_glszm_SmallAreaHighGrayLevelEmphasis'
 'pre_square_glcm_Correlation' 'pre_squareroot_firstorder_10Percentile'
 'pre_squareroot_glrlm_RunLengthNonUniformity'
 'pre_squareroot_glrlm_ShortRunLowGrayLevelEmphasis'
 'pre_squareroot_glszm_LargeAreaHighGrayLevelEmphasis'
 'pre_squareroot_ngtdm_Strength'
 'pre_wavelet-HHH_firstorder_RobustMeanAbsoluteDeviation'
 'pre_wave

In [20]:
lasso_data = feature[feature_list]
lasso_data

,pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis,pre_gradient_ngtdm_Strength,pre_log-sigma-0-5-mm-3D_firstorder_Maximum,pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized,pre_log-sigma-0-5-mm-3D_glszm_ZoneEntropy,pre_log-sigma-1-5-mm-3D_gldm_DependenceVariance,pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-1-5-mm-3D_glszm_ZoneEntropy,pre_log-sigma-2-0-mm-3D_glszm_SizeZoneNonUniformityNormalized,pre_log-sigma-2-5-mm-3D_glcm_ClusterProminence,...,pre_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,pre_wavelet-HLL_glcm_ClusterShade,pre_wavelet-LHH_firstorder_InterquartileRange,pre_wavelet-LHH_glcm_Autocorrelation,pre_wavelet-LHH_glcm_ClusterProminence,pre_wavelet-LHH_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LHL_glcm_MaximumProbability,pre_wavelet-LHL_gldm_DependenceNonUniformityNormalized,pre_wavelet-LHL_glszm_SizeZoneNonUniformityNormalized,pre_wavelet-LLL_glszm_SizeZoneNonUniformity
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.185579,0.000000,0.503490,0.500054,3.326360,32.140323,0.537190,3.095795,0.250000,0.873257,...,0.520000,-0.040668,0.014027,2.249324,0.508357,3.576389e-01,0.286262,0.067464,0.295918,1.285714
17815426,0.129629,0.515066,0.947600,0.500902,1.879965,37.869959,0.580000,2.846439,0.190083,0.864854,...,0.555556,0.383879,0.041206,2.287857,0.507338,3.125001e-01,0.293893,0.068238,0.358025,3.380952
20109646,0.132136,0.000000,0.792671,0.500003,2.563865,30.514001,0.549861,3.609384,0.226337,0.946904,...,0.663265,0.035427,0.047805,2.250002,0.494680,4.444444e-01,0.299754,0.072300,0.289941,4.709091
22320836,0.117801,0.000000,0.840529,0.500013,2.125815,30.632048,0.615912,3.559900,0.095000,1.041575,...,0.625000,0.083842,0.031887,2.261520,0.503356,2.242504e-09,0.314916,0.070319,0.413333,3.129032
32221501,0.177425,0.000000,0.919746,0.501282,2.321928,36.945214,0.680000,3.021928,0.112245,0.864144,...,0.625000,0.026187,0.020100,2.276937,0.507983,3.854167e-01,0.328967,0.062173,0.140000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,0.080184,0.558423,0.827367,0.500105,1.921928,36.737259,0.555556,2.584963,0.222222,0.888459,...,0.555556,0.068679,0.043639,2.264913,0.504439,3.125003e-01,0.308295,0.072752,0.500000,3.631579
56124435,0.106811,0.000000,1.254027,0.500080,2.500000,31.396415,0.916824,2.698598,0.156463,0.898044,...,0.520000,0.050650,0.037374,2.251903,0.496926,8.333334e-02,0.311250,0.067981,0.208333,6.652174
56131318,0.216765,0.212867,1.288005,0.500282,2.065046,35.005772,0.571938,3.728502,0.156250,0.954157,...,0.555556,0.098652,0.044196,2.247773,0.502292,4.394531e-01,0.297178,0.068513,0.157025,9.392857


In [21]:
final_data = pd.concat([lasso_data, target], axis=1)
final_data

,pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis,pre_gradient_ngtdm_Strength,pre_log-sigma-0-5-mm-3D_firstorder_Maximum,pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized,pre_log-sigma-0-5-mm-3D_glszm_ZoneEntropy,pre_log-sigma-1-5-mm-3D_gldm_DependenceVariance,pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-1-5-mm-3D_glszm_ZoneEntropy,pre_log-sigma-2-0-mm-3D_glszm_SizeZoneNonUniformityNormalized,pre_log-sigma-2-5-mm-3D_glcm_ClusterProminence,...,pre_wavelet-HLL_glcm_ClusterShade,pre_wavelet-LHH_firstorder_InterquartileRange,pre_wavelet-LHH_glcm_Autocorrelation,pre_wavelet-LHH_glcm_ClusterProminence,pre_wavelet-LHH_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LHL_glcm_MaximumProbability,pre_wavelet-LHL_gldm_DependenceNonUniformityNormalized,pre_wavelet-LHL_glszm_SizeZoneNonUniformityNormalized,pre_wavelet-LLL_glszm_SizeZoneNonUniformity,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.185579,0.000000,0.503490,0.500054,3.326360,32.140323,0.537190,3.095795,0.250000,0.873257,...,-0.040668,0.014027,2.249324,0.508357,3.576389e-01,0.286262,0.067464,0.295918,1.285714,1.0
17815426,0.129629,0.515066,0.947600,0.500902,1.879965,37.869959,0.580000,2.846439,0.190083,0.864854,...,0.383879,0.041206,2.287857,0.507338,3.125001e-01,0.293893,0.068238,0.358025,3.380952,2.0
20109646,0.132136,0.000000,0.792671,0.500003,2.563865,30.514001,0.549861,3.609384,0.226337,0.946904,...,0.035427,0.047805,2.250002,0.494680,4.444444e-01,0.299754,0.072300,0.289941,4.709091,0.0
22320836,0.117801,0.000000,0.840529,0.500013,2.125815,30.632048,0.615912,3.559900,0.095000,1.041575,...,0.083842,0.031887,2.261520,0.503356,2.242504e-09,0.314916,0.070319,0.413333,3.129032,1.0
32221501,0.177425,0.000000,0.919746,0.501282,2.321928,36.945214,0.680000,3.021928,0.112245,0.864144,...,0.026187,0.020100,2.276937,0.507983,3.854167e-01,0.328967,0.062173,0.140000,3.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,0.080184,0.558423,0.827367,0.500105,1.921928,36.737259,0.555556,2.584963,0.222222,0.888459,...,0.068679,0.043639,2.264913,0.504439,3.125003e-01,0.308295,0.072752,0.500000,3.631579,2.0
56124435,0.106811,0.000000,1.254027,0.500080,2.500000,31.396415,0.916824,2.698598,0.156463,0.898044,...,0.050650,0.037374,2.251903,0.496926,8.333334e-02,0.311250,0.067981,0.208333,6.652174,1.0
56131318,0.216765,0.212867,1.288005,0.500282,2.065046,35.005772,0.571938,3.728502,0.156250,0.954157,...,0.098652,0.044196,2.247773,0.502292,4.394531e-01,0.297178,0.068513,0.157025,9.392857,2.0


In [22]:
clinical_train = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_train.csv", index_col=0) 
clinical_test = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_test.csv", index_col=0) 
clinical_train.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_test.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_train

,encoded_PDL1
pt_no,
51127213,1.0
38924846,0.0
56241796,1.0
40815882,1.0
33908849,0.0
...,...
47181472,2.0
49087653,1.0
49588132,0.0


In [23]:
rad_features_list = final_data.columns.to_list()
rad_features_list.remove('encoded_PDL1')
rad_features_list

['pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis',
 'pre_gradient_ngtdm_Strength',
 'pre_log-sigma-0-5-mm-3D_firstorder_Maximum',
 'pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized',
 'pre_log-sigma-0-5-mm-3D_glszm_ZoneEntropy',
 'pre_log-sigma-1-5-mm-3D_gldm_DependenceVariance',
 'pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized',
 'pre_log-sigma-1-5-mm-3D_glszm_ZoneEntropy',
 'pre_log-sigma-2-0-mm-3D_glszm_SizeZoneNonUniformityNormalized',
 'pre_log-sigma-2-5-mm-3D_glcm_ClusterProminence',
 'pre_log-sigma-2-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis',
 'pre_logarithm_firstorder_Mean',
 'pre_original_glszm_SmallAreaHighGrayLevelEmphasis',
 'pre_square_glcm_Correlation',
 'pre_squareroot_firstorder_10Percentile',
 'pre_squareroot_glrlm_RunLengthNonUniformity',
 'pre_squareroot_glrlm_ShortRunLowGrayLevelEmphasis',
 'pre_squareroot_glszm_LargeAreaHighGrayLevelEmphasis',
 'pre_squareroot_ngtdm_Strength',
 'pre_wavelet-HHH_firstorder_RobustMeanAbsoluteDevi

In [24]:
clinical_train[rad_features_list] = final_data[rad_features_list]
clinical_test[rad_features_list] = final_data[rad_features_list]
clinical_train

,encoded_PDL1,pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis,pre_gradient_ngtdm_Strength,pre_log-sigma-0-5-mm-3D_firstorder_Maximum,pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized,pre_log-sigma-0-5-mm-3D_glszm_ZoneEntropy,pre_log-sigma-1-5-mm-3D_gldm_DependenceVariance,pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-1-5-mm-3D_glszm_ZoneEntropy,pre_log-sigma-2-0-mm-3D_glszm_SizeZoneNonUniformityNormalized,...,pre_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,pre_wavelet-HLL_glcm_ClusterShade,pre_wavelet-LHH_firstorder_InterquartileRange,pre_wavelet-LHH_glcm_Autocorrelation,pre_wavelet-LHH_glcm_ClusterProminence,pre_wavelet-LHH_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LHL_glcm_MaximumProbability,pre_wavelet-LHL_gldm_DependenceNonUniformityNormalized,pre_wavelet-LHL_glszm_SizeZoneNonUniformityNormalized,pre_wavelet-LLL_glszm_SizeZoneNonUniformity
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,0.182223,0.000000,0.664567,0.500122,2.654255,37.500958,0.654321,3.169925,0.140496,...,0.625000,0.018805,0.049314,2.254418,0.522630,1.914694e-01,0.382262,0.055567,0.160494,2.166667
38924846,0.0,0.114407,0.000000,1.053989,0.500798,1.446617,36.365287,0.820000,2.160964,0.135802,...,0.555556,0.126726,0.032076,2.300821,0.515161,1.971438e-07,0.367369,0.065830,0.280000,3.173913
56241796,1.0,0.194852,0.000000,1.054696,0.500187,1.773557,35.884558,0.802469,2.702904,0.259259,...,0.510204,0.056162,0.034611,2.216087,0.496474,6.112501e-02,0.308905,0.072274,0.295918,7.466667
40815882,1.0,0.102096,0.000000,0.339400,0.500083,1.842371,35.679968,0.520000,2.921928,0.333333,...,0.500000,0.079406,0.017869,2.255178,0.507457,5.000001e-01,0.301444,0.068912,0.200000,1.888889
33908849,0.0,0.138322,0.125267,1.154541,0.500087,3.244271,32.072576,0.625000,3.750000,0.134948,...,0.625000,-0.010402,0.035768,2.236788,0.526369,3.342014e-01,0.305437,0.056805,0.233560,6.660377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,0.216422,0.000000,0.984203,0.500025,2.959148,31.283605,0.500329,4.002940,0.139200,...,0.531250,0.030348,0.026093,2.257654,0.500414,5.009191e-01,0.293237,0.073869,0.278464,8.258065
49087653,1.0,0.182363,0.000000,0.963001,0.500181,2.419382,34.760465,0.680000,3.121928,0.263889,...,0.520000,0.082278,0.033252,2.258274,0.509601,3.660714e-01,0.326829,0.063523,0.481481,3.560000
49588132,0.0,0.059613,0.000000,0.755543,0.500061,2.155639,30.005843,0.570312,3.452820,0.257024,...,0.510204,0.050636,0.021004,2.260859,0.502099,3.000000e-01,0.298732,0.070605,0.382716,8.660377


In [25]:
clinical_test

,encoded_PDL1,pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis,pre_gradient_ngtdm_Strength,pre_log-sigma-0-5-mm-3D_firstorder_Maximum,pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized,pre_log-sigma-0-5-mm-3D_glszm_ZoneEntropy,pre_log-sigma-1-5-mm-3D_gldm_DependenceVariance,pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-1-5-mm-3D_glszm_ZoneEntropy,pre_log-sigma-2-0-mm-3D_glszm_SizeZoneNonUniformityNormalized,...,pre_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,pre_wavelet-HLL_glcm_ClusterShade,pre_wavelet-LHH_firstorder_InterquartileRange,pre_wavelet-LHH_glcm_Autocorrelation,pre_wavelet-LHH_glcm_ClusterProminence,pre_wavelet-LHH_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LHL_glcm_MaximumProbability,pre_wavelet-LHL_gldm_DependenceNonUniformityNormalized,pre_wavelet-LHL_glszm_SizeZoneNonUniformityNormalized,pre_wavelet-LLL_glszm_SizeZoneNonUniformity
pt_no,,,,,,,,,,,,,,,,,,,,,
49645129,1.0,0.376102,0.000000,0.784699,0.501451,1.500000,34.036836,0.625000,2.000000,0.333333,...,0.555556,0.232883,0.031832,2.287451,0.513575,3.758506e-01,0.334436,0.066412,0.187500,1.000000
47451087,2.0,0.148120,0.000000,0.909381,0.500393,2.222192,31.503150,0.847222,1.584963,0.316327,...,0.680000,0.042036,0.025827,2.250490,0.517434,2.868924e-01,0.311520,0.065401,0.421875,6.615385
50247936,2.0,0.294407,0.000000,0.663612,0.500711,2.584963,34.201741,0.722222,3.022055,0.218935,...,0.510204,-0.026707,0.049748,2.277287,0.502510,3.125001e-01,0.335819,0.061168,0.210000,2.000000
50270330,0.0,0.101676,0.000000,1.111858,0.500537,2.355389,32.239879,0.644970,2.815072,0.125000,...,0.520000,0.023399,0.023895,2.270106,0.511055,3.333334e-01,0.293375,0.067630,0.333333,3.080000
46695156,0.0,0.207729,0.103788,1.392923,0.495069,2.584963,33.298747,0.471074,2.845351,0.166667,...,0.259259,-0.342654,0.047780,2.330411,0.545628,3.875387e-01,0.302848,0.061680,0.173469,3.666667
42522605,1.0,0.243123,0.000000,0.772831,0.500370,3.022055,37.302017,0.537190,3.277613,0.324444,...,0.555556,0.080702,0.023051,2.288136,0.513092,9.421169e-02,0.314198,0.066585,0.180000,5.388889
43986877,0.0,0.120215,0.000000,0.808700,0.500086,1.584963,34.687723,0.625000,2.000000,0.156250,...,0.680000,0.033015,0.024663,2.267666,0.507291,1.311039e-07,0.320760,0.066140,0.440000,2.833333
52779802,1.0,0.241519,0.000000,0.578099,0.500499,2.321928,31.560712,0.500000,2.000000,0.250000,...,0.500000,0.008499,0.029737,2.273946,0.500733,3.333335e-01,0.301087,0.060895,0.333333,1.000000
43876141,1.0,0.105462,0.000000,0.722494,0.500195,2.299896,30.481053,0.506173,2.419382,0.351562,...,0.625000,0.094011,0.019513,2.247884,0.508523,1.785714e-01,0.288080,0.063107,0.265306,1.428571


In [26]:
train_data = clinical_train
test_data = clinical_test

In [27]:
# train_data, test_data = train_test_split(final_data, test_size=0.2, shuffle=True, stratify=target, random_state=2021)

In [28]:
train_data

,encoded_PDL1,pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis,pre_gradient_ngtdm_Strength,pre_log-sigma-0-5-mm-3D_firstorder_Maximum,pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized,pre_log-sigma-0-5-mm-3D_glszm_ZoneEntropy,pre_log-sigma-1-5-mm-3D_gldm_DependenceVariance,pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-1-5-mm-3D_glszm_ZoneEntropy,pre_log-sigma-2-0-mm-3D_glszm_SizeZoneNonUniformityNormalized,...,pre_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,pre_wavelet-HLL_glcm_ClusterShade,pre_wavelet-LHH_firstorder_InterquartileRange,pre_wavelet-LHH_glcm_Autocorrelation,pre_wavelet-LHH_glcm_ClusterProminence,pre_wavelet-LHH_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LHL_glcm_MaximumProbability,pre_wavelet-LHL_gldm_DependenceNonUniformityNormalized,pre_wavelet-LHL_glszm_SizeZoneNonUniformityNormalized,pre_wavelet-LLL_glszm_SizeZoneNonUniformity
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,0.182223,0.000000,0.664567,0.500122,2.654255,37.500958,0.654321,3.169925,0.140496,...,0.625000,0.018805,0.049314,2.254418,0.522630,1.914694e-01,0.382262,0.055567,0.160494,2.166667
38924846,0.0,0.114407,0.000000,1.053989,0.500798,1.446617,36.365287,0.820000,2.160964,0.135802,...,0.555556,0.126726,0.032076,2.300821,0.515161,1.971438e-07,0.367369,0.065830,0.280000,3.173913
56241796,1.0,0.194852,0.000000,1.054696,0.500187,1.773557,35.884558,0.802469,2.702904,0.259259,...,0.510204,0.056162,0.034611,2.216087,0.496474,6.112501e-02,0.308905,0.072274,0.295918,7.466667
40815882,1.0,0.102096,0.000000,0.339400,0.500083,1.842371,35.679968,0.520000,2.921928,0.333333,...,0.500000,0.079406,0.017869,2.255178,0.507457,5.000001e-01,0.301444,0.068912,0.200000,1.888889
33908849,0.0,0.138322,0.125267,1.154541,0.500087,3.244271,32.072576,0.625000,3.750000,0.134948,...,0.625000,-0.010402,0.035768,2.236788,0.526369,3.342014e-01,0.305437,0.056805,0.233560,6.660377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,0.216422,0.000000,0.984203,0.500025,2.959148,31.283605,0.500329,4.002940,0.139200,...,0.531250,0.030348,0.026093,2.257654,0.500414,5.009191e-01,0.293237,0.073869,0.278464,8.258065
49087653,1.0,0.182363,0.000000,0.963001,0.500181,2.419382,34.760465,0.680000,3.121928,0.263889,...,0.520000,0.082278,0.033252,2.258274,0.509601,3.660714e-01,0.326829,0.063523,0.481481,3.560000
49588132,0.0,0.059613,0.000000,0.755543,0.500061,2.155639,30.005843,0.570312,3.452820,0.257024,...,0.510204,0.050636,0.021004,2.260859,0.502099,3.000000e-01,0.298732,0.070605,0.382716,8.660377


In [29]:
print('Data for Modeling: ' + str(train_data.shape))
print('Unseen Data For Predictions: ' + str(test_data.shape))

Data for Modeling: (79, 38)
Unseen Data For Predictions: (22, 38)


In [30]:
ypCR_clf = setup(data=train_data,
            test_data=test_data,
            target='encoded_PDL1',
            normalize=True, 
            normalize_method='zscore', 
            data_split_shuffle=True, 
            data_split_stratify='encoded_PDL1',
            fold = 5,
            fold_strategy = 'stratifiedkfold',
            fold_shuffle = True, 
            session_id=3457, 
            use_gpu=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 750, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that

,Description,Value
0,Session id,3457
1,Target,encoded_PDL1
2,Target type,Multiclass
3,Original data shape,"(101, 38)"
4,Transformed data shape,"(101, 38)"
5,Transformed train set shape,"(79, 38)"
6,Transformed test set shape,"(22, 38)"
7,Numeric features,37
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 750, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that

In [31]:
train = get_config(variable="train_transformed")
train['encoded_PDL1'].value_counts()

1.0    36
0.0    22
2.0    21
Name: encoded_PDL1, dtype: int64

In [32]:
test = get_config(variable="test_transformed")
test['encoded_PDL1'].value_counts()

1.0    10
2.0     6
0.0     6
Name: encoded_PDL1, dtype: int64

In [33]:
compare_models(n_select=3, sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.6050,0.7554,0.6050,0.6418,0.6031,0.3866,0.3934,0.0420
lda,Linear Discriminant Analysis,0.5308,0.6835,0.5308,0.5623,0.5234,0.2877,0.2969,0.0280
et,Extra Trees Classifier,0.4933,0.6501,0.4933,0.5216,0.4696,0.1693,0.1791,0.1900
rf,Random Forest Classifier,0.5050,0.6470,0.5050,0.5486,0.4931,0.2011,0.2111,0.2240
knn,K Neighbors Classifier,0.5058,0.6341,0.5058,0.5563,0.4902,0.1998,0.2119,0.0620
ada,Ada Boost Classifier,0.4917,0.6215,0.4917,0.4575,0.4513,0.1513,0.1567,0.0880
nb,Naive Bayes,0.4033,0.5956,0.4033,0.4072,0.3809,0.0894,0.0926,0.0280
lightgbm,Light Gradient Boosting Machine,0.3925,0.5855,0.3925,0.4357,0.3955,0.0563,0.0583,0.5140
gbc,Gradient Boosting Classifier,0.3783,0.5366,0.3783,0.4092,0.3708,0.0264,0.0264,0.3900
dt,Decision Tree Classifier,0.3800,0.5199,0.3800,0.3649,0.3679,0.0405,0.0395,0.0260


Processing:   0%|          | 0/63 [00:00<?, ?it/s]

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=3457, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                            priors=None, shrinkage=None, solver='svd',
                            store_covariance=False, tol=0.0001),
 ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=None, max_features='sqrt',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=3

In [34]:
lightgbm = create_model('lightgbm', fold=5, round=4, cross_validation=True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3750,0.5495,0.3750,0.3958,0.3768,0.0476,0.0488
1,0.4375,0.6624,0.4375,0.4667,0.4336,0.0943,0.0991
2,0.3750,0.5866,0.3750,0.3854,0.3735,0.0588,0.0599
3,0.3750,0.5729,0.3750,0.5083,0.3982,0.0184,0.0193
4,0.4000,0.5561,0.4000,0.4222,0.3956,0.0625,0.0643
Mean,0.3925,0.5855,0.3925,0.4357,0.3955,0.0563,0.0583
Std,0.0245,0.0406,0.0245,0.0459,0.0214,0.0245,0.0258


In [35]:
prediction = predict_model(lightgbm, data=test_data)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.5455,0.6771,0.5455,0.5284,0.4965,0.2361,0.2657
